# Ejemplo de Quantization
---

En este ejemplo vamos a ver como cambiar la representación del modelo pasando los pesos y activaciones de FP32 a INT8. De esta forma, se obtinenen dos beneficios potenciales:


1.   Reducimos el tamaño que ocupa el modelo ya que los pesos ocupan una cuarta parte (8 bits vs 32 bits por peso).
2.   Si el dispositivo incorpora hardware para trabajar en 8 bits, se reduce el tiempo de ejecución. Sino, se mantiene el mismo que para 32 bits.

---

## 1. Instalar e importar las librerías necesarias

En este ejemplo vamos a trabajar con Pytorch y los modelos de torchvision

In [1]:
!pip3 install torch torchvision torchinfo numpy

In [2]:
from torchvision.models import resnet18, ResNet18_Weights, resnet50, ResNet50_Weights
from torchinfo import summary
import torch
import torchvision
import time
import numpy as np

## 2. Definir el modelo

Definimos el modelo, en este caso, usamos AlexNet pre-entrenada en ImageNet. Usamos esta red ya que es una red lineal sin conexiones residuales que producen problemas con la cuantización. Este tipo de problemas se pueden solventar cambiando algunas operaciones del modelo como se ve en este ejemplo para ResNet50 (https://github.com/zanvari/resnet50-quantization/blob/main/quantization-resnet50.ipynb).

In [18]:
model = torchvision.models.alexnet(weights=torchvision.models.AlexNet_Weights)
preprocessing = ResNet18_Weights.IMAGENET1K_V1.transforms()
summary(model, input_size=(1, 3, 224, 224))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:05<00:00, 48.6MB/s]


Layer (type:depth-idx)                   Output Shape              Param #
AlexNet                                  [1, 1000]                 --
├─Sequential: 1-1                        [1, 256, 6, 6]            --
│    └─Conv2d: 2-1                       [1, 64, 55, 55]           23,296
│    └─ReLU: 2-2                         [1, 64, 55, 55]           --
│    └─MaxPool2d: 2-3                    [1, 64, 27, 27]           --
│    └─Conv2d: 2-4                       [1, 192, 27, 27]          307,392
│    └─ReLU: 2-5                         [1, 192, 27, 27]          --
│    └─MaxPool2d: 2-6                    [1, 192, 13, 13]          --
│    └─Conv2d: 2-7                       [1, 384, 13, 13]          663,936
│    └─ReLU: 2-8                         [1, 384, 13, 13]          --
│    └─Conv2d: 2-9                       [1, 256, 13, 13]          884,992
│    └─ReLU: 2-10                        [1, 256, 13, 13]          --
│    └─Conv2d: 2-11                      [1, 256, 13, 13]         

## 3. Definir un data loader

Una vez descargados los datos, tenemos que crear un DataLoader de Pytorch para poder usarlos con nuestro modelo.


In [5]:
dataset = torchvision.datasets.CIFAR10(root='./cifar10', train=True, transform=preprocessing, download=True)
train_data_loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)

100%|██████████| 170498071/170498071 [00:03<00:00, 43691184.03it/s]


Extracting ./cifar10/cifar-10-python.tar.gz to ./cifar10


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## 4. Preparar la cuantización

En este ejemplo vamos a usar una 'Quantization-Aware Training' para calibrar y transformar los pesos y activaciones de FP32 a INT8. De esta forma, los pesos se adaptan al nuevo rango de representación evitando problemas de cálculos que se salen fuera de rango y obteniendo un mejor accruacy que usando otras técnicas de cuantización como el 'Post-Training Quantization'.

Para ello, tenemos que añadir unos adaptadores a la entrada y salida del modelo para convertir las entradas de FP32 a INT8 y nuestras salidas de INT8 a FP32. Tras esto, definimos la librería que realizará la cuantización y que depende del hardware en el que vamos a desplegar. Pytorch ofrece las siguientes opciones: https://pytorch.org/docs/stable/quantization.html#backend-hardware-support

In [19]:
model_fp32 = torch.nn.Sequential(torch.quantization.QuantStub(), model, torch.quantization.DeQuantStub())
model_fp32.eval()
model_fp32.qconfig = torch.quantization.get_default_qat_qconfig('qnnpack')
model_fp32_prepared = torch.quantization.prepare_qat(model_fp32.train())
summary(model_fp32_prepared, input_size=(1, 3, 224, 224))

Layer (type:depth-idx)                                       Output Shape              Param #
Sequential                                                   [1, 1000]                 --
├─QuantStub: 1-1                                             [1, 3, 224, 224]          --
│    └─FusedMovingAvgObsFakeQuantize: 2-1                    [1, 3, 224, 224]          --
├─AlexNet: 1-2                                               [1, 1000]                 --
│    └─Sequential: 2-2                                       [1, 256, 6, 6]            --
│    │    └─Conv2d: 3-1                                      [1, 64, 55, 55]           23,296
│    │    └─ReLU: 3-2                                        [1, 64, 55, 55]           --
│    │    └─MaxPool2d: 3-3                                   [1, 64, 27, 27]           --
│    │    └─Conv2d: 3-4                                      [1, 192, 27, 27]          307,392
│    │    └─ReLU: 3-5                                        [1, 192, 27, 27]         

## 5. Entrenamiento del modelo

Realizamos unas épocas para calibrar los pesos del modelo y adaptarlo a la nueva representación. Para ello, usamos la base de datos que hemos descargado en el punto 3.

In [20]:
n_epochs = 1
opt = torch.optim.Adam(model_fp32_prepared.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
model_fp32_prepared.train()
for epoch in range(n_epochs): # Entrenamos n epocas
    train_running_loss = 0.0
    train_running_correct = 0
    counter = 0
    time_start = time.time()
    for inputs, labels in train_data_loader: # Obtenemos todos los batch de entrenamiento y los usamos para entrenar
        inputs = inputs.to('cuda')
        labels = labels.to('cuda')
        opt.zero_grad()
        outs = model_fp32_prepared(inputs)
        loss = loss_fn(outs, labels)
        train_running_loss += loss.item()
        _, preds = torch.max(outs.data, 1)
        train_running_correct += (preds == labels).sum().item()
        counter = counter + 1
        loss.backward()
        opt.step()

    epoch_loss = train_running_loss / counter
    epoch_acc = 100. * (train_running_correct / len(train_data_loader.dataset))
    time_end = time.time() - time_start
    print(f'** Summary for epoch {epoch}: '
		f'loss: {epoch_loss:#.3g}, acc: {epoch_acc:#.3g}]  '
		f'time: {time_end:.3f}s **')

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


** Summary for epoch 0: loss: 1.02e+07, acc: 10.0]  time: 137.871s **


## 6. Exportar el modelo en INT8

Una vez que hemos realizado el entrenamiento para pasar a INT8, simplemente limpiamos las capas auxiliares que añade Pytorch para realizar la calibración y exportamos el modelo a TorchScript para poder usarlo en un móvil.

In [21]:
model_fp32_prepared.eval().cpu()
model_int8 = torch.quantization.convert(model_fp32_prepared, inplace=True).cpu()
model_int8_script = torch.jit.script(model_int8) # Export to TorchScript
torch.jit.save(model_int8_script, './model_int8.pt')

Además, vamos a realizar una  inferencia de prueba para analizar el rendimiento del modelo inicial y el cuantizado.

In [23]:
image = torch.Tensor(np.random.rand(1,3,224,244)).float().cpu()
print(image.shape)

# FP32
model.cpu()
time_start = time.time()
model(image)
time_end = time.time() - time_start
print(f'Execution time of the fp32 model: {time_end:.3f}s')

# INT8
time_start = time.time()
model_int8(image)
time_end = time.time() - time_start
print(f'Execution time of the int8 model: {time_end:.3f}s')

torch.Size([1, 3, 224, 244])
Execution time of the fp32 model: 0.129s
Execution time of the int8 model: 0.035s
